In [1]:
import os

from estnltk import Text, Layer

In [2]:
with open("vallakohus_esimene\Harju_Hageri_Kohila_id1346_1888a.ann", encoding="UTF8") as annotated_file:
    annotation = annotated_file.read()
with open("vallakohus_esimene\Harju_Hageri_Kohila_id1346_1888a.txt", encoding="UTF8") as txt:
    textfile = txt.read()

In [3]:
print(annotation)

T2	Isik 38 49	Juhan Kõerw
T4	Isik 88 101	Kustas Wäljar
T5	KO_koht 164 178	Rabiwere mõisa
T1	KO_koht 16 30	Wäljapere koha
T3	KO_koht 64 78	Rabiwere mõisa
T6	Isik 222 235	Kustas Wäljar
T7	Isik 300 313	Kustas Wäljar
T8	KO_koht 503 512	Wäljapere
T9	Isik 564 577	Kustas Wäljar
T10	Isik 695 708	Kustas Wäljar



In [4]:
textfile = textfile.replace(".", ". ")
text = Text(textfile)
text

text
"Kohtu ette tuli Wäljapere koha omanik Juhan Kõerw ja kaebas: et Rabiwere mõisa teu mees Kustas Wäljar olla tema hobused (kolm hoost) tema karja maalt ära ajanud ja Rabiwere mõisa teu majade juure aja külge kinni sidunud. Kustas Wäljar sai ette kutsutud ja küsitud, miks ta hobused sinna oli ajanud? Kustas Wäljar ütles: Mina ei ajanud mitte hobusid sinna, waid hobused oliwad ise sinna tulnud, mina nägin neid seal ja käskisin metsa wahti neid kinni sidud, mida ka metsawaht minu käsu peale tegi. Kui Wäljapere sulane hobusid tahtnud sealt ära wia, siis ei olla Kustas Wäljar neid mitte tahtnud lasta ära wia, waid nõudnud 3 (kolm) rubla selle eest, et ta neid olla öö läbi seal walwanud. Et Kustas Wäljar ilma miski pohjuseta hobusid keelas ära wiimast, mõistis kohus temale 1 (üks) rubla trahwi. Kohus on mõistetud ja trahw maksetud. """


In [5]:
layer = Layer(name="gold_ner", text_object=text, attributes=['gold_nertag'])

In [6]:
annotation_dictionary = {}

splitted_annotation = annotation.split("\n")
for row in splitted_annotation:
    splitted_row = row.split("\t")
    if (len(splitted_row) == 3):
        trigger = splitted_row[0]
        location = splitted_row[1]
        entity = splitted_row[2]
        annotation_dictionary[trigger] = [location, entity]
    else:
        continue

In [7]:
for key in annotation_dictionary:
    value = annotation_dictionary.get(key)
    ner, loc = value[0].split(" ")[0], (value[0].split(" ")[1], value[0].split(" ")[2])
    if ner == "Isik":
        layer.add_annotation((int(loc[0]), int(loc[1])), gold_nertag="PER")
    if ner == "KO_koht":
        layer.add_annotation((int(loc[0]), int(loc[1])), gold_nertag="LOC")
text.add_layer(layer)

In [8]:
text.gold_ner

Layer(name='gold_ner', attributes=('gold_nertag',), spans=SL[Span('Wäljapere koha', [{'gold_nertag': 'LOC'}]),
Span('Juhan Kõerw', [{'gold_nertag': 'PER'}]),
Span('Rabiwere mõisa', [{'gold_nertag': 'LOC'}]),
Span('Kustas Wäljar', [{'gold_nertag': 'PER'}]),
Span('Rabiwere mõisa', [{'gold_nertag': 'LOC'}]),
Span('Kustas Wäljar', [{'gold_nertag': 'PER'}]),
Span('Kustas Wäljar', [{'gold_nertag': 'PER'}]),
Span('Wäljapere', [{'gold_nertag': 'LOC'}]),
Span('Kustas Wäljar', [{'gold_nertag': 'PER'}]),
Span('Kustas Wäljar', [{'gold_nertag': 'PER'}])])